<a href="https://colab.research.google.com/github/PranavSingla122/Age-Predictor/blob/main/Agepredictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/Train.csv'
df = pd.read_csv(file_path)



#DATA SET
#https://www.kaggle.com/datasets/abdullah0a/human-age-prediction-synthetic-dataset/data?select=Train.csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#TRAINING


import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

data = pd.read_csv('Train.csv')

# Split 'Blood Pressure (s/d)' into two columns: Systolic and Diastolic
data[['Systolic Pressure', 'Diastolic Pressure']] = data['Blood Pressure (s/d)'].str.split('/', expand=True).astype(float)
data = data.drop(columns=['Blood Pressure (s/d)'])

# Split features and target
X = data.drop(columns=['Age (years)'])
y = data['Age (years)']

# Define categorical and numerical columns
categorical_columns = ['Gender', 'Smoking Status', 'Alcohol Consumption', 'Diet',
                       'Chronic Diseases', 'Medication Use', 'Family History',
                       'Mental Health Status', 'Sleep Patterns', 'Education Level',
                       'Income Level', 'Physical Activity Level']  # Include Physical Activity

numerical_columns = X.columns.difference(categorical_columns)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a ColumnTransformer for preprocessing both numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),  # Scale numerical columns
        ('cat', OneHotEncoder(), categorical_columns)  # One-hot encode categorical columns
    ])

# Preprocess the data
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)

# Define the neural network model
model = Sequential()

# Add layers
model.add(Dense(64, input_dim=X_train_preprocessed.shape[1], activation='relu'))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1))  # Output layer (for regression, predicting a single value)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train_preprocessed, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model on the test set
test_loss = model.evaluate(X_test_preprocessed, y_test)
print(f"Test Loss: {test_loss}")

# Make predictions on the test set
predictions = model.predict(X_test_preprocessed)


In [ ]:
#TESTING


import pandas as pd
import numpy as np

# Load new data
new_data = pd.read_csv('Test.csv')

# Ensure new data has the same structure as training data
new_data[['Systolic Pressure', 'Diastolic Pressure']] = new_data['Blood Pressure (s/d)'].str.split('/', expand=True).astype(float)
new_data = new_data.drop(columns=['Blood Pressure (s/d)'])

# Extract features (same as training features)
X_new = new_data

# Preprocess the new data
X_new_preprocessed = preprocessor.transform(X_new)

# Make predictions
predictions = model.predict(X_new_preprocessed)

# Convert predictions to integer part only
predictions_int = np.floor(predictions).astype(int)

# Add the integer predictions as a new column to the new_data DataFrame
new_data['Age'] = predictions_int

# Create a DataFrame with Gender and Age columns
# Assuming 'Gender' is a column in the original 'new_data'
result_df = new_data[['Gender', 'Age']]

# Set pandas display options to show all rows
pd.set_option('display.max_rows', None)

# Print the resulting DataFrame
print(result_df)

# Optionally reset the display option to default if needed later
pd.reset_option('display.max_rows')
